## Day3-01: Agent Session 会话管理

5 天 AI Agents 强化课程的第三天第一部分，我们将学习如何管理 Agent 的会话和状态。

你将学会：

- 什么是 Session 以及如何在 Agent 中使用它
- 如何构建有状态的 Agent
- 如何将会话持久化到数据库
- 上下文压缩（Context Compaction）技术
- Session State 的最佳实践

## 精简总结：什么是 Session？

### 核心概念

- **问题**：LLM 本身是"失忆"的，每次请求都是独立的
- **解决方案**：使用 Session（会话）来维持对话上下文

### Session 的两个核心组件

```
Session
  |-- Events (事件列表)
  |     |-- 用户输入
  |     |-- Agent 响应
  |     |-- 工具调用
  |     |-- 工具返回结果
  |
  |-- State (状态字典)
        |-- key: value 键值对
        |-- 所有子 Agent 和工具可访问
```

### 两种存储方式

| 服务 | 特点 | 适用场景 |
|------|------|----------|
| InMemorySessionService | 内存存储，重启丢失 | 开发测试 |
| DatabaseSessionService | 持久化存储 | 生产环境 |

本次课程直播回放请到 youtube 观看: [youtube](https://www.youtube.com/playlist?list=PLqFaTIg4myu9r7uRoNfbJhHUbLp-1t1YE)

---
## Section 1: 设置环境

### 1.1 配置你的 Gemini API 密钥

In [23]:
import os
from pathlib import Path

# 读取项目根目录的 .env 文件
env_file = Path.cwd().parent / '.env'

if env_file.exists():
    for line in env_file.read_text().splitlines():
        if line.startswith('GOOGLE_API_KEY='):
            os.environ["GOOGLE_API_KEY"] = line.split('=', 1)[1].strip()
            print("Gemini API key 配置完成。")
            break
else:
    print(f"请在以下路径创建 .env 文件: {env_file}")

Gemini API key 配置完成。


### 1.2 导入 ADK 组件

In [24]:
from typing import Any, Dict

from google.adk.agents import Agent, LlmAgent
from google.adk.apps.app import App, EventsCompactionConfig
from google.adk.models.google_llm import Gemini
from google.adk.sessions import DatabaseSessionService
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools.tool_context import ToolContext
from google.genai import types

print("ADK 组件导入成功。")

ADK 组件导入成功。


### 1.3 辅助函数

创建一个辅助函数来管理会话。它支持单个查询或多个查询的顺序处理。

In [25]:
# 定义整个 notebook 中复用的辅助函数
async def run_session(
    runner_instance: Runner,
    user_queries: list[str] | str = None,
    session_name: str = "default",
):
    print(f"\n ### 会话: {session_name}")

    # 从 Runner 获取应用名称
    app_name = runner_instance.app_name

    # 尝试创建新会话或获取现有会话
    try:
        session = await session_service.create_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
    except:
        session = await session_service.get_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )

    # 如果提供了查询则处理
    if user_queries:
        # 将单个查询转换为列表以统一处理
        if type(user_queries) == str:
            user_queries = [user_queries]

        # 顺序处理列表中的每个查询
        for query in user_queries:
            print(f"\n用户 > {query}")

            # 将查询字符串转换为 ADK Content 格式
            query = types.Content(role="user", parts=[types.Part(text=query)])

            # 异步流式输出 Agent 响应
            async for event in runner_instance.run_async(
                user_id=USER_ID, session_id=session.id, new_message=query
            ):
                # 检查事件是否包含有效内容
                if event.content and event.content.parts:
                    # 过滤掉空的或 "None" 响应
                    if (
                        event.content.parts[0].text != "None"
                        and event.content.parts[0].text
                    ):
                        print(f"{MODEL_NAME} > ", event.content.parts[0].text)
    else:
        print("没有查询！")


print("辅助函数定义完成。")

辅助函数定义完成。


### 1.4 配置重试选项

配置请求重试策略，处理速率限制或临时服务不可用等瞬时错误。

In [26]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # 最大重试次数
    exp_base=7,  # 延迟乘数
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # 遇到这些 HTTP 错误时重试
)

---
## Section 2: 会话管理基础

### 2.1 问题背景

大型语言模型（LLM）的核心特性是**无状态**的。它们的认知仅限于你在单次 API 调用中提供的信息。这意味着没有适当上下文管理的 Agent 只会对当前提示做出反应，而不考虑任何历史记录。

**为什么这很重要？** 想象一下，和一个每句话后就忘记你所说一切的人交谈。这就是我们使用原始 LLM 时面临的挑战！

在 ADK 中，我们使用：
- `Session` 进行**短期记忆管理**
- `Memory` 进行**长期记忆**（下一个 notebook 学习）

### 2.2 什么是 Session？

#### Session（会话）

Session 是对话的容器。它按时间顺序封装对话历史，并记录**单次连续对话**中的所有工具交互和响应。Session 绑定到特定用户和 Agent；不与其他用户共享。同样，一个 Agent 的会话历史不会与其他 Agent 共享。

在 ADK 中，**Session** 由两个关键组件组成：`Events` 和 `State`：

**Events（事件）**：

> Session 是对话的容器，而 `Events` 是对话的构建块。
>
> 事件类型示例：
> - *用户输入*：来自用户的消息（文本、音频、图像等）
> - *Agent 响应*：Agent 对用户的回复
> - *工具调用*：Agent 决定使用外部工具或 API
> - *工具输出*：工具调用返回的数据，Agent 用它继续推理

**State（状态）**：

> `session.state` 是 Agent 的草稿本，用于存储和更新对话期间需要的动态细节。可以把它想象成一个全局 `{key, value}` 键值对存储，所有子 Agent 和工具都可以访问。

![Session 状态和事件](https://storage.googleapis.com/github-repo/kaggle-5days-ai/day3/session-state-and-events.png)

### 2.3 如何管理 Session？

一个 Agent 应用可以有多个用户，每个用户可能与应用有多个会话。为了管理这些会话和事件，ADK 提供了 **Session Manager** 和 **Runner**。

1. **`SessionService`**：存储层
   - 管理会话数据的创建、存储和检索
   - 不同的实现满足不同需求（内存、数据库、云）

2. **`Runner`**：编排层
   - 管理用户和 Agent 之间的信息流
   - 自动维护对话历史
   - 在后台处理上下文工程


### 2.4 实现我们的第一个有状态 Agent

让我们构建第一个有状态的 Agent，它可以记住并进行有建设性的对话。

ADK 提供不同类型的会话，适合不同需求。首先，我们从简单的会话管理选项（`InMemorySessionService`）开始：

In [28]:
APP_NAME = "default"  # 应用名称
USER_ID = "default"   # 用户 ID
SESSION = "default"   # 会话 ID

MODEL_NAME = "gemini-2.5-flash-lite"


# 步骤 1：创建 LLM Agent
root_agent = Agent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="text_chat_bot",
    description="一个文本聊天机器人",  # Agent 用途描述
)

# 步骤 2：设置会话管理
# InMemorySessionService 将对话存储在 RAM 中（临时的）
session_service = InMemorySessionService()

# 步骤 3：创建 Runner
runner = Runner(agent=root_agent, app_name=APP_NAME, session_service=session_service)

print("有状态 Agent 初始化完成！")
print(f"   - 应用: {APP_NAME}")
print(f"   - 用户: {USER_ID}")
print(f"   - 使用: {session_service.__class__.__name__}")

有状态 Agent 初始化完成！
   - 应用: default
   - 用户: default
   - 使用: InMemorySessionService


### 2.5 测试有状态 Agent

现在让我们看看 Session 的魔力！

In [29]:
# 在同一个会话中运行两个查询的对话
# 注意：两个查询都是同一个会话的一部分，所以上下文得以保持
await run_session(
    runner,
    [
        "你好，我是小明！美国的首都是什么？",
        "你好！我的名字是什么？",  # 这次，Agent 应该记得！
    ],
    "stateful-agentic-session",
)


 ### 会话: stateful-agentic-session

用户 > 你好，我是小明！美国的首都是什么？
gemini-2.5-flash-lite >  你好，小明！美国的首都是华盛顿哥伦比亚特区（Washington, D.C.）。

用户 > 你好！我的名字是什么？
gemini-2.5-flash-lite >  你好！你的名字是小明。


**成功！** Agent 记住了你的名字，因为两个查询都是同一个会话的一部分。Runner 自动维护了对话历史。

但有个问题：`InMemorySessionService` 是临时的。**一旦应用停止，所有对话历史都会丢失。**

### （可选）2.6 测试 Agent 的遗忘

> 要验证 Agent 会忘记对话，**重启内核**。然后，**运行本 notebook 之前的所有单元格，但不要运行 2.5 中的 `run_session`。**
>
> 现在运行下面的单元格。你会发现 Agent 不记得之前对话中的任何内容。

In [30]:
# 重启内核后运行此单元格。所有历史都会消失...
await run_session(
    runner,
    ["我之前问了你什么？", "提醒我一下，我的名字是什么？"],
    "stateful-agentic-session",
)  # 注意，我们使用相同的会话名称


 ### 会话: stateful-agentic-session

用户 > 我之前问了你什么？
gemini-2.5-flash-lite >  你之前问了我“美国的首都是什么？”

用户 > 提醒我一下，我的名字是什么？
gemini-2.5-flash-lite >  你的名字是小明。


### 问题

会话信息不是持久的（即有意义的对话会丢失）。虽然这在测试环境中很方便，但**在现实世界中，用户应该能够引用过去的对话并恢复会话。** 要实现这一点，我们必须持久化信息。

---
## Section 3: 使用 DatabaseSessionService 实现持久化会话

虽然 `InMemorySessionService` 非常适合原型开发，但现实世界的应用需要对话能够在重启、崩溃和部署后存活。让我们升级到持久化存储！

### 3.1 选择合适的 SessionService

ADK 为不同需求提供不同的 SessionService 实现：

| 服务 | 用例 | 持久性 | 最适合 |
|------|------|--------|--------|
| **InMemorySessionService** | 开发和测试 | 重启后丢失 | 快速原型 |
| **DatabaseSessionService** | 自管理应用 | 重启后保留 | 中小型应用 |
| **Agent Engine Sessions** | GCP 生产环境 | 完全托管 | 企业级规模 |

### 3.2 实现持久化会话

让我们升级到使用 SQLite 的 `DatabaseSessionService`。这让我们在本演示中无需单独的数据库服务器就能获得持久性。

创建一个能够与用户进行对话的 `chatbot_agent`。

In [31]:
# 步骤 1：创建相同的 Agent（注意这次我们使用 LlmAgent）
chatbot_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="text_chat_bot",
    description="一个具有持久化记忆的文本聊天机器人",
)

# 步骤 2：切换到 DatabaseSessionService
# SQLite 数据库会自动创建
db_url = "sqlite:///my_agent_data.db"  # 本地 SQLite 文件
session_service = DatabaseSessionService(db_url=db_url)

# 步骤 3：使用持久化存储创建新的 Runner
runner = Runner(agent=chatbot_agent, app_name=APP_NAME, session_service=session_service)

print("已升级到持久化会话！")
print(f"   - 数据库: my_agent_data.db")
print(f"   - 会话将在重启后保留！")

已升级到持久化会话！
   - 数据库: my_agent_data.db
   - 会话将在重启后保留！


### 3.3 测试运行 1：验证持久性

在第一次测试运行中，我们将使用会话 ID `test-db-session-01` 开始一个新对话。首先介绍我们的名字是 'Sam' 然后问一个问题。在第二轮，我们会询问 Agent 我们的名字。

由于我们使用的是 `DatabaseSessionService`，Agent 应该记得这个名字。

对话之后，我们将直接检查 `my_agent_data.db` SQLite 数据库，看看对话 `events`（用户查询和模型响应）是如何存储的。

In [32]:
await run_session(
    runner,
    ["你好，我是小明！美国的首都是什么？", "你好！我的名字是什么？"],
    "test-db-session-01",
)


 ### 会话: test-db-session-01

用户 > 你好，我是小明！美国的首都是什么？
gemini-2.5-flash-lite >  你好，小明！美国的首都是华盛顿哥伦比亚特区。

用户 > 你好！我的名字是什么？
gemini-2.5-flash-lite >  你好！你的名字是小明。


### （可选）3.4 测试运行 2：恢复对话

> 现在，让我们再次重复测试，但这次，**停止这个 Notebook 的内核并重新启动它。**
>
> 1. 运行 notebook 中之前的所有单元格，**但不要运行**之前 Section 3.3 中的 `run_session` 单元格。
>
> 2. 现在，使用**相同的会话 ID**（`test-db-session-01`）运行下面的单元格。

我们会问一个新问题，然后再次询问我们的名字。**因为会话是从数据库加载的，Agent 应该仍然记得**第一次测试运行中我们的名字是 'Sam'。这展示了持久化会话的强大之处。

In [34]:
await run_session(
    runner,
    ["中国的首都是什么？", "你好！我的名字是什么？"],
    "test-db-session-01",
)


 ### 会话: test-db-session-01

用户 > 中国的首都是什么？
gemini-2.5-flash-lite >  中国的首都是北京。

用户 > 你好！我的名字是什么？
gemini-2.5-flash-lite >  你好！你的名字是小明。


### 3.5 验证会话数据是隔离的

如前所述，会话是 Agent 和用户之间的私人对话（即两个会话不共享信息）。让我们用不同的会话名称 `test-db-session-02` 运行 `run_session` 来确认这一点。

In [35]:
await run_session(
    runner, ["你好！我的名字是什么？"], "test-db-session-02"
)  # 注意，我们使用新的会话名称


 ### 会话: test-db-session-02

用户 > 你好！我的名字是什么？
gemini-2.5-flash-lite >  你好！根据我现有的记忆，我没有被告知你的名字。请问我可以如何称呼你呢？


### 3.6 事件是如何存储在数据库中的？

由于我们使用 SQLite 数据库存储信息，让我们快速查看一下信息是如何存储的。

In [37]:
import sqlite3

def check_data_in_db():
    with sqlite3.connect("my_agent_data.db") as connection:
        cursor = connection.cursor()
        result = cursor.execute(
            "select app_name, session_id, author, content from events"
        )
        print([_[0] for _ in result.description])
        for each in result.fetchall():
            print(each)


check_data_in_db()

['app_name', 'session_id', 'author', 'content']
('default', 'test-db-session-01', 'user', '{"parts": [{"text": "\\u4f60\\u597d\\uff0c\\u6211\\u662f\\u5c0f\\u660e\\uff01\\u7f8e\\u56fd\\u7684\\u9996\\u90fd\\u662f\\u4ec0\\u4e48\\uff1f"}], "role": "user"}')
('default', 'test-db-session-01', 'text_chat_bot', '{"parts": [{"text": "\\u4f60\\u597d\\uff0c\\u5c0f\\u660e\\uff01\\u7f8e\\u56fd\\u7684\\u9996\\u90fd\\u662f\\u534e\\u76db\\u987f\\u54e5\\u4f26\\u6bd4\\u4e9a\\u7279\\u533a\\u3002"}], "role": "model"}')
('default', 'test-db-session-01', 'user', '{"parts": [{"text": "\\u4f60\\u597d\\uff01\\u6211\\u7684\\u540d\\u5b57\\u662f\\u4ec0\\u4e48\\uff1f"}], "role": "user"}')
('default', 'test-db-session-01', 'text_chat_bot', '{"parts": [{"text": "\\u4f60\\u597d\\uff01\\u4f60\\u7684\\u540d\\u5b57\\u662f\\u5c0f\\u660e\\u3002"}], "role": "model"}')
('default', 'test-db-session-01', 'user', '{"parts": [{"text": "\\u5370\\u5ea6\\u7684\\u9996\\u90fd\\u662f\\u4ec0\\u4e48\\uff1f"}], "role": "user"}')
('defau

---
## Section 4: 上下文压缩（Context Compaction）

如你所见，所有事件都完整地存储在会话数据库中，这会快速累积。对于长时间的复杂任务，事件列表可能变得非常大，导致性能下降和成本增加。

但如果我们能自动总结过去呢？让我们使用 ADK 的**上下文压缩**功能来看看**如何自动减少存储在 Session 中的上下文**。

![上下文压缩](https://storage.googleapis.com/github-repo/kaggle-5days-ai/day3/context-compaction.png)

### 4.1 为 Agent 创建 App

要启用此功能，我们使用在 Section 3.2 中创建的相同 `chatbot_agent`。

第一步是创建一个名为 `App` 的对象。我们给它一个名称并传入我们的 chatbot_agent。

我们还将创建一个新配置来进行上下文压缩。这个 **`EventsCompactionConfig`** 定义了两个关键变量：

- **compaction_interval**：要求 Runner 每 `n` 次对话后压缩历史
- **overlap_size**：定义保留多少之前的对话用于重叠

然后我们将这个 app 提供给 Runner。

In [39]:
# 重新定义启用了事件压缩的 app
research_app_compacting = App(
    name="research_app_compacting",
    root_agent=chatbot_agent,
    # 这是新增的部分！
    events_compaction_config=EventsCompactionConfig(
        compaction_interval=3,  # 每 3 次调用触发压缩
        overlap_size=1,  # 保留 1 个之前的轮次作为上下文
    ),
)

db_url = "sqlite:///my_agent_data.db"  # 本地 SQLite 文件
session_service = DatabaseSessionService(db_url=db_url)

# 为升级后的 app 创建新的 Runner
research_runner_compacting = Runner(
    app=research_app_compacting, session_service=session_service
)


print("Research App 已升级为带有事件压缩功能！")

Research App 已升级为带有事件压缩功能！


/var/folders/65/wyr5mgy15d5_v2j0l5pxxm840000gn/T/ipykernel_51688/894108061.py:6: UserWarning: [EXPERIMENTAL] EventsCompactionConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  events_compaction_config=EventsCompactionConfig(


### 4.2 运行演示

现在，让我们进行一个足够长的对话来触发压缩。当你运行下面的单元格时，输出看起来像普通对话。但是，因为我们配置了 `App`，压缩过程会在第 3 次调用后在后台静默运行。

在下一步，我们将证明它确实发生了。

In [40]:
# 第 1 轮
await run_session(
    research_runner_compacting,
    "AI 在医疗保健领域的最新消息是什么？",
    "compaction_demo",
)

# 第 2 轮
await run_session(
    research_runner_compacting,
    "药物发现有什么新进展吗？",
    "compaction_demo",
)

# 第 3 轮 - 压缩应该在这轮之后触发！
await run_session(
    research_runner_compacting,
    "告诉我更多关于你发现的第二个进展。",
    "compaction_demo",
)

# 第 4 轮
await run_session(
    research_runner_compacting,
    "参与其中的主要公司有哪些？",
    "compaction_demo",
)


 ### 会话: compaction_demo

用户 > AI 在医疗保健领域的最新消息是什么？
gemini-2.5-flash-lite >  AI 在医疗保健领域的最新消息主要集中在以下几个方面：

**1. 疾病诊断和药物研发的加速：**

*   **更精准的影像诊断：** AI 在分析医学影像（如 X 光、CT、MRI）方面取得了巨大进展，能够更早、更准确地检测出肿瘤、视网膜病变、心血管疾病等，有时甚至能发现人类医生难以察觉的细微病灶。例如，有研究表明 AI 在检测乳腺癌方面已能达到甚至超越放射科医生。
*   **个性化药物研发：** AI 正在被用于加速新药的发现和开发过程，通过分析大量的基因组学、蛋白质组学数据，预测潜在的药物靶点，设计新的分子结构，并预测药物的疗效和副作用。这有望显著缩短药物研发周期，降低成本。
*   **预测性诊断：** AI 模型能够分析患者的电子健康记录、基因信息、生活习惯等数据，预测其罹患某种疾病的风险，从而实现早期干预和预防。

**2. 改善患者护理和管理：**

*   **虚拟健康助手和聊天机器人：** AI 驱动的聊天机器人可以为患者提供初步的健康咨询、解答常见问题、提醒用药、预约医生等，减轻医疗系统的压力，提高患者的可及性。
*   **远程医疗和监测：** AI 可以分析可穿戴设备收集的生理数据（如心率、睡眠、活动量），实时监测患者的健康状况，并在出现异常时发出警报，尤其对于慢性病患者和老年人具有重要意义。
*   **个性化治疗方案：** AI 能够根据患者的具体情况，如基因组特征、病情进展、过往治疗反应等，推荐最适合的治疗方案，实现真正的个性化医疗。

**3. 提升医疗系统效率：**

*   **自动化行政流程：** AI 可以自动化处理大量的行政任务，如病历录入、账单处理、排班管理等，从而解放医护人员，让他们更专注于患者护理。
*   **优化医院运营：** AI 可以分析医院的运营数据，预测患者流量，优化资源配置（如床位、设备、人员），提高整体运营效率。
*   **辅助手术：** AI 驱动的机器人辅助手术系统能够提供更精准的操作、更小的创伤和更快的恢复，同时 AI 也可以在手术过程中为外科医生提供实时指导和建议。

**4. 挑战与发展方向：**

尽管 AI 在医疗保健领域的应用前景

### 4.3 验证会话历史中的压缩

上面的对话看起来很正常，但历史已经在幕后被改变了。我们如何证明它？

我们可以检查会话中的 `events` 列表。压缩过程**不会删除旧事件；它用包含摘要的单个新 `Event` 替换它们。** 让我们找到它。

In [41]:
# 获取最终的会话状态
final_session = await session_service.get_session(
    app_name=research_runner_compacting.app_name,
    user_id=USER_ID,
    session_id="compaction_demo",
)

print("--- 搜索压缩摘要事件 ---")
found_summary = False
for event in final_session.events:
    # 压缩事件有 'compaction' 属性
    if event.actions and event.actions.compaction:
        print("\n成功！找到了压缩事件：")
        print(f"  作者: {event.author}")
        print(f"\n 压缩后的信息: {event}")
        found_summary = True
        break

if not found_summary:
    print(
        "\n未找到压缩事件。尝试增加演示中的轮次数量。"
    )

--- 搜索压缩摘要事件 ---

成功！找到了压缩事件：
  作者: user

 压缩后的信息: model_version=None content=None grounding_metadata=None partial=None turn_complete=None finish_reason=None error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=None live_session_resumption_update=None input_transcription=None output_transcription=None avg_logprobs=None logprobs_result=None cache_metadata=None citation_metadata=None invocation_id='23ebeb47-eb54-42ec-a04e-18ed2c4825fe' author='user' actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}, requested_tool_confirmations={}, compaction={'start_timestamp': 1764492174.69086, 'end_timestamp': 1764492185.991522, 'compacted_content': {'parts': [{'function_call': None, 'code_execution_result': None, 'executable_code': None, 'file_data': None, 'function_response': None, 'inline_data': None, 'text': "The conversation history shows a user asking about the lates

### 4.4 你完成了什么：自动上下文管理

你刚刚找到了证据！会话历史中那个特殊摘要 `Event` 的存在就是压缩过程的具体结果。

**让我们回顾一下你刚刚见证的：**

1. **静默操作**：你运行了一个标准对话，从外部看没有任何不同。
2. **后台压缩**：因为你用 `EventsCompactionConfig` 配置了 `App`，ADK `Runner` 自动监控对话长度。一旦达到阈值，它就在后台触发摘要过程。
3. **验证结果**：通过检查会话的事件，你找到了 LLM 生成的摘要。这个摘要现在替换了 Agent 活动上下文中更冗长的旧轮次。

**对于这个对话中所有未来的轮次，Agent 将收到这个简洁的摘要而不是完整的历史。** 这节省了成本，提高了性能，并帮助 Agent 专注于最重要的内容。

### 4.5 ADK 中更多上下文工程选项

#### 自定义压缩
在这个例子中，我们使用了 ADK 的默认摘要器。对于更高级的用例，你可以通过定义自定义 `SlidingWindowCompactor` 并将其传递给配置来提供你自己的。这允许你控制摘要提示，甚至使用不同的专门 LLM 来完成任务。你可以在[官方文档](https://google.github.io/adk-docs/context/compaction/)中阅读更多内容。

#### 上下文缓存
ADK 还提供**上下文缓存**来帮助通过缓存请求数据来减少馈送给 LLM 的静态指令的 token 大小。在[这里](https://google.github.io/adk-docs/context/caching/)阅读更多内容。

### 问题

虽然我们可以进行上下文压缩并使用数据库恢复会话，但现在我们面临新的挑战。在某些情况下，**我们有想要跨其他会话共享的关键信息或偏好。**

在这些场景中，与其共享整个会话历史，从几个关键变量传输信息可以改善会话体验。让我们看看如何做到这一点！

---
## Section 5: 使用 Session State

### 5.1 为 Session State 管理创建自定义工具

让我们探索如何通过自定义工具手动管理会话状态。在这个例子中，我们将识别一个**可传输的特征**，如用户的名字和他们的国家，并创建工具来捕获和保存它。

**为什么用这个例子？**

用户名是一个完美的信息示例：

- 只介绍一次但多次引用
- 应该在整个对话中持久存在
- 代表一个增强个性化的用户特定特征

这里，为了演示目的，我们将创建两个工具，可以从 Session State 存储和检索用户名和国家。**注意所有工具都可以访问 `ToolContext` 对象。** 你不必为每条想要共享的信息创建单独的工具。

In [16]:
# 为状态键定义作用域级别（遵循最佳实践）
USER_NAME_SCOPE_LEVELS = ("temp", "user", "app")


# 这演示了工具如何使用 tool_context 写入会话状态。
# 'user:' 前缀表示这是用户特定的数据。
def save_userinfo(
    tool_context: ToolContext, user_name: str, country: str
) -> Dict[str, Any]:
    """
    记录和保存用户名和国家到会话状态的工具。

    Args:
        user_name: 要存储在会话状态中的用户名
        country: 用户国家的名称
    """
    # 使用 'user:' 前缀写入会话状态，表示用户数据
    tool_context.state["user:name"] = user_name
    tool_context.state["user:country"] = country

    return {"status": "success"}


# 这演示了工具如何从会话状态读取。
def retrieve_userinfo(tool_context: ToolContext) -> Dict[str, Any]:
    """
    从会话状态检索用户名和国家的工具。
    """
    # 从会话状态读取
    user_name = tool_context.state.get("user:name", "未找到用户名")
    country = tool_context.state.get("user:country", "未找到国家")

    return {"status": "success", "user_name": user_name, "country": country}


print("工具创建完成。")

工具创建完成。


**关键概念：**
- 工具可以访问 `tool_context.state` 来读/写会话状态
- 使用描述性的键前缀（`user:`、`app:`、`temp:`）进行组织
- 状态在同一会话的对话轮次之间持久存在

### 5.2 创建带有 Session State 工具的 Agent

现在让我们创建一个可以访问我们的会话状态管理工具的新 Agent：

In [42]:
# 配置
APP_NAME = "default"
USER_ID = "default"
MODEL_NAME = "gemini-2.5-flash-lite"

# 创建带有会话状态工具的 Agent
root_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="text_chat_bot",
    description="""一个文本聊天机器人。
    管理用户上下文的工具：
    * 当提供用户名和国家时使用 `save_userinfo` 工具来记录。
    * 当需要获取用户名和国家时使用 `retrieve_userinfo` 工具。
    """,
    tools=[save_userinfo, retrieve_userinfo],  # 向 Agent 提供工具
)

# 设置会话服务和 Runner
session_service = InMemorySessionService()
runner = Runner(agent=root_agent, session_service=session_service, app_name="default")

print("带有会话状态工具的 Agent 初始化完成！")

带有会话状态工具的 Agent 初始化完成！


### 5.3 测试 Session State 实战

让我们测试 Agent 如何使用会话状态在对话轮次之间记住信息：

In [43]:
# 演示会话状态的测试对话
await run_session(
    runner,
    [
        "你好，今天怎么样？我的名字是什么？",  # Agent 还不应该知道名字
        "我的名字是小明。我来自中国。",  # 提供名字 - Agent 应该保存它
        "我的名字是什么？我来自哪个国家？",  # Agent 应该从会话状态中回忆
    ],
    "state-demo-session",
)


 ### 会话: state-demo-session

用户 > 你好，今天怎么样？我的名字是什么？
gemini-2.5-flash-lite >  你好！我很好。你想知道你的名字吗？

用户 > 我的名字是小明。我来自中国。



用户 > 我的名字是什么？我来自哪个国家？


### 5.4 检查 Session State

让我们直接检查会话状态，看看存储了什么：

In [44]:
# 检索会话并检查其状态
session = await session_service.get_session(
    app_name=APP_NAME, user_id=USER_ID, session_id="state-demo-session"
)

print("会话状态内容：")
print(session.state)
print("\n注意 'user:name' 和 'user:country' 键存储了我们的数据！")

会话状态内容：
{'user:name': '小明', 'user:country': '中国'}

注意 'user:name' 和 'user:country' 键存储了我们的数据！


### 5.5 Session State 隔离

我们已经看到，会话状态的一个重要特性是它按会话隔离。让我们通过开始一个新会话来演示这一点：

In [45]:
# 开始一个全新的会话 - Agent 不会知道我们的名字
await run_session(
    runner,
    ["你好，今天怎么样？我的名字是什么？"],
    "new-isolated-session",
)

# 预期：Agent 不会知道名字，因为这是一个不同的会话


 ### 会话: new-isolated-session

用户 > 你好，今天怎么样？我的名字是什么？
gemini-2.5-flash-lite >  你好！我很好。你想让我怎么称呼你？


### 5.6 跨会话状态共享

虽然会话默认是隔离的，但你可能会注意到一些有趣的事情。让我们检查新会话（`new-isolated-session`）的状态：

In [46]:
# 检查新会话的状态
session = await session_service.get_session(
    app_name=APP_NAME, user_id=USER_ID, session_id="new-isolated-session"
)

print("新会话状态：")
print(session.state)

# 注意：根据实现，你可能会在这里看到共享状态。
# 这就是会话特定状态和用户特定状态之间区别变得重要的地方。

新会话状态：
{'user:name': '小明', 'user:country': '中国'}


---
## 清理

In [47]:
# 清理任何现有的数据库以重新开始（如果 Notebook 重启）
import os

if os.path.exists("my_agent_data.db"):
    os.remove("my_agent_data.db")
print("已清理旧的数据库文件")

已清理旧的数据库文件


---
## 总结

恭喜！你已经学会了构建有状态 AI Agent 的基础知识：

- **上下文工程** - 你理解了如何使用上下文压缩为 LLM 组装上下文
- **Sessions 和 Events** - 你可以在多个轮次中维护对话历史
- **持久化存储** - 你知道如何让对话在重启后存活
- **Session State** - 你可以在对话期间跟踪结构化数据
- **手动状态管理** - 你体验了手动方法的强大和局限性
- **生产考虑** - 你已准备好处理现实世界的挑战

## 恭喜！你完成了 Session 会话管理的学习！

### 更多学习资源

参考以下文档了解更多：

- [ADK 文档](https://google.github.io/adk-docs/)
- [ADK Sessions](https://google.github.io/adk-docs/)
- [ADK Session-State](https://medium.com/google-cloud/2-minute-adk-manage-context-efficiently-with-artifacts-6fcc6683d274)
- [ADK Session Compaction](https://google.github.io/adk-docs/context/compaction/#define-compactor)

### 下一步 - 长期记忆系统（Part 2）

#### 为什么需要 Memory？

在这个 notebook 中，我们手动识别了几个特征（用户名和国家）并构建了工具来管理它们。但真实的对话涉及数百个这样的特征：
- 用户偏好和习惯
- 过去的交互及其结果
- 领域知识和专业水平
- 沟通风格和模式
- 主题之间的上下文关系

**ADK 中的 Memory 系统自动化了整个过程**，使其成为构建真正上下文感知 Agent 的宝贵资产，可以满足任何用户当前和未来的需求。

在下一个 notebook（Part 2：Memory 管理）中，你将学习：
- 启用从对话中自动提取记忆
- 构建随时间学习和适应的 Agent
- 大规模创建真正个性化的体验
- 管理跨会话的长期知识

准备好将你的手动状态管理转变为智能、自动化的 Memory 系统了吗？让我们继续 Part 2！